In [3]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/AURA2/data/Final_result_list.pkl','rb') as f:
  Final_result_list = pickle.load(f)

In [4]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/AURA2/data/X_train.pkl','rb') as f:
  X_train = pickle.load(f)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### FastText 불러오기

In [5]:
!pip install --upgrade gensim==3.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [6]:
# RAM초과;;;
from gensim import models

ko_model = models.fasttext.load_facebook_model('/content/drive/MyDrive/Colab Notebooks/FastText_pretrained/cc.ko.300.bin')

ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa1\x9c'; replacing invalid characters, using '\\xed\\xa1\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb0\x80'; replacing invalid characters, using '\\xed\\xb0\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa7\x80'; replacing invalid characters, using '\\xed\\xa7\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb8\xb0'; replacing invalid characters, using '\\xed\\xb8\\xb0'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa0\x9c'; replacing invalid characters, using '\\xed\\xa0\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa6\xac'; replacing invalid characters, using '\\xed\\xa6\\xac'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb3\xb5'; replacing invalid characters, using '\\xed\\xb

In [7]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

#### 문장의 임베딩 평균을 이용한 유사도

In [8]:
# ner사용하지 않고, 단순히 문장 간 유사도 비교
def fasttext_sim(convrs, model) : # model : FastText model
  cos_sim_lst = []
  sentences = []
  for i in range(0, len(convrs[0]), 2) :
    if len(convrs[0]) % 2 == 0 : # 개수가 짝수
      sent1 = convrs[0][i]
      sent2 = convrs[0][i+1]
      sentences.append(sent1)
      sentences.append(sent2)

    else :
      if i == len(convrs[0])-1 :
        sentences.append(convrs[0][i])
        break
      else :
        sent1 = convrs[0][i]
        sent2 = convrs[0][i+1]
        sentences.append(sent1)
        sentences.append(sent2)
  
  sentences_embedding = []
  for i in range(len(sentences)) :
    tmp = 0
    for j in range(len(sentences[i])) :
      #print(model[sentences[i][j]])
      #if np.isnan(model[sentences[i][j]]) :
      #  continue
      #else : 
      tmp += model[sentences[i][j]]
    if len(sentences[i]) == 0 :
      return [0], 0
    sentences_embedding.append(tmp / len(sentences[i])) # 단어 임베딩들을 모두 합한 후 평균낸다.
  #print(sentences_embedding)
  
  # sentences간 cos_sim 구하기
  for i in range(len(sentences_embedding) - 1) :
    #print(sentences_embedding[i+1])
    cos_sim_lst.append(cos_sim(sentences_embedding[i], sentences_embedding[i+1]))
  if len(cos_sim_lst) == 0 :
    return [0], 0
  return cos_sim_lst, sum(cos_sim_lst)/len(cos_sim_lst)

In [9]:
# 문장1에서 등장한 ner이 문장2에서 얼마나 등장하는지
def ner_count(convrs) :
  result = []
  ners = []
  for i in range(0, len(convrs[0]), 2) :
    if len(convrs[0]) % 2 == 0 : # 개수가 짝수
      ner1 = convrs[1][i]
      ner2 = convrs[1][i+1]
      ners.append(ner1)
      ners.append(ner2)
    else :
      if i == len(convrs[0])-1 :
        ners.append(convrs[1][i])
        break
      else :
        ner1 = convrs[1][i]
        ner2 = convrs[1][i+1]
        ners.append(ner1)
        ners.append(ner2)
  
  ners_true = [] # 의미있는 ner들만. 즉, B-I / PAD / O 등을 다 제외하고
  for i in range(len(ners)) :
    tmp = []
    for j in range(len(ners[i])) :
      if ners[i][j] != 'PAD' and ners[i][j] != 'O' :
        tmp.append(ners[i][j][2:]) # 태그에서 B-I 제거
    
    tmp = list(set(tmp)) # 중복 제거
    ners_true.append(tmp)
  
  #print(ners_true)
  for i in range(len(ners_true) - 1) : # 문장1과 문장2에서 동일하게 등장하는 태그의 개수를 센다.
    cnt = 0
    for j in range(len(ners_true[i])) :
      for k in range(len(ners_true[i+1])) :
        if ners_true[i][j] == ners_true[i+1][k] :
          cnt += 1
    result.append(cnt)
  
  return sum(result)

In [10]:
# test
fasttext_sim(Final_result_list[0], ko_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


([0.50635445, 0.74727, 0.73110384], 0.6615760922431946)

In [11]:
ner_count(Final_result_list[6])

3

In [12]:
# fasstext_sim과 ner_count를 특정 가중치를 통해 곱한다
convrs_sim = []
for i in range(len(Final_result_list)) :
  ft_sim = fasttext_sim(Final_result_list[i], ko_model)[1]
  ner_cnt = ner_count(Final_result_list[i])
  res = 0.6 * ft_sim + 0.4 * ner_cnt
  convrs_sim.append(res)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


In [13]:
len(convrs_sim)

10000

In [ ]:
convrs_sim

In [ ]:
convrs_sim_tuple = []
for i in range(len(convrs_sim)) :
  convrs_sim_tuple.append((convrs_sim[i], i))
convrs_sim_tuple

In [21]:
convrs_sim_tuple_asc = sorted(convrs_sim_tuple, key = lambda x : x[0])
convrs_sim_tuple_asc[:-5:-1]

[(17.618294764829404, 9210),
 (7.23204252201578, 9018),
 (5.663914484136245, 9170),
 (4.9182853400707245, 9473)]

In [25]:
def extract_max_min_n(X, sim, n = 5) :
  min_idx = [s[1] for s in convrs_sim_tuple_asc[:n]]
  max_idx = [s[1] for s in convrs_sim_tuple_asc[-1:-1-n:-1]]

  max_lst = [X[i] for i in max_idx]
  min_lst = [X[i] for i in min_idx]

  return max_lst, min_lst 

In [26]:
max_convrs, min_convrs = extract_max_min_n(X_train, convrs_sim_tuple_asc)

In [28]:
min_convrs

[[['아', '데이', '터', '없어서', '오늘', '아침', '에', '동숲', '못함'],
  ['아놬', '너', '케이', '티임?'],
  ['나', 'skt', 'n', 'ㅜㅜ'],
  ['ㅜㅜ',
   'kt',
   '면',
   '데이',
   '터',
   '보내주려',
   '했는데',
   '나',
   '맨날',
   '집에서',
   '수업',
   '들어서',
   '데이',
   '터',
   '남아돌아'],
  ['까니', '아', '좋겠다'],
  ['한', '달에', '2기가', '인데'],
  ['난', '맨날', '밖에서', '동숲', '해서', '데이', '터', '부족', '한', '듯'],
  ['계속', '이월되어가지고', '지금', '6기가인', '가', '있음'],
  [],
  []],
 [['다른',
   '애들',
   '은',
   '헐~',
   '~!!',
   '면접',
   '언제지??',
   '하고',
   '그러는데',
   '그냥',
   '말해줬음',
   '아~~',
   '그래요??',
   '하면',
   '될',
   '것을',
   '왜',
   '토',
   '달아'],
  [],
  ['ㅡㅡ'],
  ['씨', '혹시', '스터디', '장', '내려놓고', '싶나', '요', '(칼)'],
  [],
  [],
  [],
  ['야', '근데', '왜', '이렇게', '반응', '이'],
  ['내가', '제일', '싫어', '하는', '임이', '임'],
  ['아니꼬워'],
  ['그니까'],
  ['오픈', '채팅방', '에', '쓰면'],
  ['쟤', '나한테만', '아니꼬운', '것', '같아'],
  ['20', '대', '남자', '로', '알아볼', '듯'],
  [],
  []],
 [[],
  [],
  [],
  ['스승', '의', '날', '에디션'],
  ['오늘', '스승', '의', '날이여?', '환장', '해'],
  ['바지', '원래'